# Inicialização

In [2]:
from datetime import datetime
datetime.now().strftime("%Y-%m-%d %H:%M:%S")

'2023-03-30 17:15:31'

In [6]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import snscrape.modules.twitter as sntwitter
from tqdm import tqdm

# Oculta warnings do notebook
# import warnings
# warnings.filterwarnings('ignore')

# Configurações de visualização
pd.set_option('display.max_columns', 30)
pd.options.display.float_format = '{:,.2f}'.format

In [10]:
dataset_path = '../data/raw/'
dataset_name = 'consulta_cand_2022_SP.csv'
cand_sp_df = pd.read_csv(f"{dataset_path}{dataset_name}", sep=';')
print(cand_sp_df.shape)
cand_sp_df.info()

(3659, 71)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3659 entries, 0 to 3658
Data columns (total 71 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   DT_GERACAO                     3659 non-null   object 
 1   HH_GERACAO                     3659 non-null   object 
 2   ANO_ELEICAO                    3659 non-null   int64  
 3   CD_TIPO_ELEICAO                3659 non-null   int64  
 4   NM_TIPO_ELEICAO                3659 non-null   object 
 5   NR_TURNO                       3659 non-null   int64  
 6   CD_ELEICAO                     3659 non-null   int64  
 7   DS_ELEICAO                     3659 non-null   object 
 8   DT_ELEICAO                     3659 non-null   object 
 9   TP_ABRANGENCIA                 3659 non-null   object 
 10  SG_UF                          3659 non-null   object 
 11  SG_UE                          3659 non-null   object 
 12  NM_UE                          3659 n

In [ ]:
# Carregando dataset com os dados de redes sociais dos candidatos de SP nas eleições de 2022
rs_df = pd.read_csv(f"{dataset_path}rede_social_candidato_2022_SP.csv", sep=';')
print(rs_df.shape)
rs_df.info()

## Juntando os datasets

In [ ]:
# Filtrando somente candidatos com conta no Twitter
rs_twitter_df = rs_df[rs_df.DS_URL.str.contains('twitter', case=False)]
print(rs_twitter_df.shape)

In [ ]:
# Juntando as duas bases
df = pd.merge(cand_sp_df, rs_twitter_df[['SQ_CANDIDATO', 'DS_URL']], on="SQ_CANDIDATO", how="left")
df.shape

In [ ]:
# Filtrando somente candidatos a deputado estadual
candidates_df = df.loc[
    (df['DS_CARGO'] == 'DEPUTADO ESTADUAL')
]

print(candidates_df.shape)

# Checagem dos dados

## Dados duplicados

In [ ]:
# Vamos checar os candidatos duplicados
duplicated_candidates_id = (candidates_df.SQ_CANDIDATO.value_counts() > 1).loc[lambda x: x == True].index
candidates_df[candidates_df['SQ_CANDIDATO'].isin(duplicated_candidates_id)][['SQ_CANDIDATO', 'DS_URL']]

In [ ]:
# Vamos checar estas contas no Twitter
drop_list = []

# A conta DrMarcioAurelio está repetida, portanto vamos remover uma das linhas
drop_list.append(180)

# A conta raiztrabalhista é de um coletivo político e não do candidato
drop_list.append(181)

# A conta Mantiqueira18 é de um coletivo político e não do candidato
drop_list.append(579)

# A conta eniotatto13114 não existe mais
drop_list.append(886)

# A conta FabioAdonis é pessoal e não posta desde 2017
drop_list.append(2344)

# A conta alefrota77 não existe mais
drop_list.append(2688)

# A conta GeraldóNobrega não existe mais
drop_list.append(2784)

# A conta rodrigo_pt13 não tem nenhum tweet
drop_list.append(3442)

# A conta valverdemogi possui apenas um tweet de 2021
drop_list.append(3443)

In [ ]:
# Removendo as contas acima
candidates_df = candidates_df.drop(drop_list)

# Exploração dos dados

In [ ]:
# Vamos criar diferentes segmentações, para facilitar a comparação de diversidade
elected_values = ['ELEITO POR QP', 'ELEITO POR MÉDIA', 'ELEITO']

# Deputados estaduais eleitos
elected_df = candidates_df.loc[
    (candidates_df['DS_SIT_TOT_TURNO'].isin(elected_values))
]

# Deputados estaduais eleitos com perfil no Twitter
twitter_df = elected_df.loc[
    (elected_df['DS_URL'].notna())
]

print(candidates_df.shape)
print(elected_df.shape)
print(twitter_df.shape)

### Analisando a diversidade

In [ ]:
print('Candidatos a deputado estadual')
print(candidates_df.groupby(['DS_GENERO', 'DS_COR_RACA']).size())
print()
print('Candidatos a deputado estadual eleitos')
print(elected_df.groupby(['DS_GENERO', 'DS_COR_RACA']).size())
print()
print('Candidatos a deputado estadual eleitos com conta no Twitter')
print(twitter_df.groupby(['DS_GENERO', 'DS_COR_RACA']).size())
print()

In [ ]:
# Visualizando a distribuição de gênero em raça, entre o candidatos, os eleitos e os eleitos com Twitter
fig, ax = plt.subplots(1, 3, figsize=(16, 4), squeeze=False)
fig.autofmt_xdate(rotation=45)

sns.histplot(
    data=candidates_df,
    hue='DS_GENERO',
    x='DS_COR_RACA',
    multiple='stack',
    ax=ax[0][0],
)

sns.histplot(
    data=elected_df,
    hue='DS_GENERO',
    x='DS_COR_RACA',
    multiple='stack',
    ax=ax[0][1],
)

sns.histplot(
    data=twitter_df,
    hue='DS_GENERO',
    x='DS_COR_RACA',
    multiple='stack',
    ax=ax[0][2],
)

print('Candidatos a Deputado Estadual ; Deputados Estaduais Eleitos ; Deputados Estaduais Eleitos com Twitter')
plt.show()

In [ ]:
# Visualizando a distribuição de gênero e raça, entre o candidatos, os eleitos e os eleitos com Twitter
fig, ax = plt.subplots(1, 3, figsize=(16, 4), squeeze=False)
fig.autofmt_xdate(rotation=45)

sns.histplot(
    data=candidates_df,
    hue='DS_COR_RACA',
    x='DS_GENERO',
    multiple='stack',
    ax=ax[0][0],
)

sns.histplot(
    data=elected_df,
    hue='DS_COR_RACA',
    x='DS_GENERO',
    multiple='stack',
    ax=ax[0][1],
)

sns.histplot(
    data=twitter_df,
    hue='DS_COR_RACA',
    x='DS_GENERO',
    multiple='stack',
    ax=ax[0][2],
)

print('Candidatos a Deputado Estadual ; Deputados Estaduais Eleitos ; Deputados Estaduais Eleitos com Twitter')
plt.show()

# Extração de dados do Twitter

## Dados das contas

In [ ]:
# Vamos listar as contas no twitter dos deputados estaduais eleitos
twitter_df.DS_URL

In [8]:
isinstance(123, str)

False

In [7]:
type(123) == str

False

In [ ]:
# Como precisamos somente do username, precisamos aplicar uma função
def get_twitter_username(link):
    link = link.rstrip('/')
    username = link.split('/')[-1]
    username = username.split('?')[0]
    return username.lower()

twitter_df['TW_USER'] = twitter_df.DS_URL.apply(get_twitter_username)
twitter_df['TW_USER'].describe()

In [ ]:
%%time
# Vamos raspar os dados numéricos de cada uma das contas do Twitter
usernames = list(twitter_df['TW_USER'])

user_data = {}
for i, username in enumerate(usernames):
    try:
        last_tweet = next(sntwitter.TwitterProfileScraper(username).get_items())
        user_data[username] = {
            'followersCount': last_tweet.user.followersCount,
            'friendsCount': last_tweet.user.friendsCount,
            'statusesCount': last_tweet.user.statusesCount,
            'favouritesCount': last_tweet.user.favouritesCount,
            'listedCount': last_tweet.user.listedCount,
            'mediaCount': last_tweet.user.mediaCount,
        }
        print(f'{i+1}/{len(usernames)} {username}: {user_data[username]}')
    except Exception as e:
        print(f'{i+1}/{len(usernames)} {username}: Erro {e}')
        user_data[username] = {
            'followersCount': 0,
            'friendsCount': 0,
            'statusesCount': 0,
            'favouritesCount': 0,
            'listedCount': 0,
            'mediaCount': 0,
        }

In [ ]:
# Agora vamos adicionar os dados dos usuários no mesmo dataframe

# Quantidade de seguidores da conta
twitter_df['TW_followersCount'] = twitter_df.TW_USER.apply(lambda x: user_data[x]['followersCount'])

# Quantidade de usuarios que a conta segue
twitter_df['TW_friendsCount'] = twitter_df.TW_USER.apply(lambda x: user_data[x]['friendsCount'])

# Quantidade de tweets postados pela conta
twitter_df['TW_statusesCount'] = twitter_df.TW_USER.apply(lambda x: user_data[x]['statusesCount'])

# Quantidade de tweets curtidos pela conta
twitter_df['TW_favouritesCount'] = twitter_df.TW_USER.apply(lambda x: user_data[x]['favouritesCount'])

twitter_df['TW_listedCount'] = twitter_df.TW_USER.apply(lambda x: user_data[x]['listedCount'])
twitter_df['TW_mediaCount'] = twitter_df.TW_USER.apply(lambda x: user_data[x]['mediaCount'])

twitter_df.head()

## Contagem de tweets

In [ ]:
%%time
# Vamos contar quantos posts e respostas cada candidato teve no período analisado (setembro e outubro de 2022)
# Esta etapa pode demorar bastante
since = '2022-09-01'
until = '2022-11-01'

user_tweets = {}
for i, username in enumerate(usernames):
    try:
        query = f'from:{username} since:{since} until:{until}'
        user_scrapping_results = sntwitter.TwitterSearchScraper(query).get_items()
        tweets = []
        for tweet in user_scrapping_results:
            tweets.append(tweet)

        user_tweets[username] = {
            'posts': tweets,
            'count': len(tweets),
        }
        print(f'{i+1}/{len(usernames)} {username}: {len(tweets)} tweets')
    except Exception as e:
        print(f'{i+1}/{len(usernames)} {username}: Erro {e}')
        user_tweets[username] = {
            'posts': [],
            'count': 0,
        }

In [ ]:
twitter_df['TW_electionTweets'] = twitter_df.TW_USER.apply(lambda x: user_tweets[x]['count'])
twitter_df['TW_electionTweets'].describe()

In [ ]:
twitter_df.columns

In [ ]:
twitter_df[twitter_df["TW_electionTweets"] > twitter_df.TW_electionTweets.median()].sort_values(
    by='TW_electionTweets', ascending=False
)[[
    'TW_USER', 'DS_GENERO', 'DS_COR_RACA', 'SG_PARTIDO', 'TW_followersCount', 'TW_statusesCount', 'TW_electionTweets'
]]

## Exploração dos dados do Twitter

In [ ]:
sns.histplot(
    data=twitter_df.loc[twitter_df['TW_electionTweets'] > 50],
    x='TW_electionTweets',
    hue='DS_GENERO',
    multiple='dodge',
    kde=True,
)

In [ ]:
sns.histplot(
    data=twitter_df.loc[twitter_df['TW_followersCount'] < 100000],
    hue='DS_GENERO',
    x='TW_followersCount',
    multiple='dodge',
    kde=True,
)

In [ ]:
sns.histplot(
    data=twitter_df.loc[twitter_df['TW_followersCount'] < 100000],
    hue='DS_COR_RACA',
    x='TW_followersCount',
    multiple='dodge',
    kde=True,   
)

# Exportação de dados

In [ ]:
# Exportando dados scrapeados

output_path = 'datasets/scrapped/'
output_file = f'deputados-estaduais-sp-{datetime.now().isoformat()}.csv'

# Para salvar os dados, descomente as linhas abaixo
# twitter_df.to_csv(f"{output_path}{output_file}", sep=';', encoding='utf-8', index=False)